In [1]:
import shapely
from shapely.geometry import MultiPoint, Point, Polygon
import shapefile
import pandas as pd
import numpy as np

In [2]:
def get_us_border_polygon():
    sf = shapefile.Reader("tl_2019_us_county.dbf")
    shapes = sf.shapes()
    fields = sf.fields
    records = sf.records()
    county_polygons = {}
    for i, record in enumerate(records):
        county = record[3]  #[3] for 5 digit FIPS CODE of county, while using record[4] indicates the name of county
        points = shapes[i].points
        poly = Polygon(points)
        county_polygons[county] = poly

    return county_polygons

In [3]:
def in_us(lat,lon):
    p = Point(lon,lat)
    for county, poly in county_poly.items():
        if poly.contains(p):
            return county
        elif poly.intersects(p):
            return county
    return None

In [5]:
county_poly = get_us_border_polygon()

In [218]:
df = pd.read_csv('C:/Users/feihu/Desktop/geo/geo_1992.csv',
                 dtype={"STATE_CODE_001": str ,"COUNTY_CODE_003" : str },
                 low_memory=False).fillna(0)

In [219]:
df['COUNTY_CODE_003']= pd.to_numeric(df['COUNTY_CODE_003'],errors='coerce')
df["COUNTY_CODE_003"] = df["COUNTY_CODE_003"].astype(int)
df["STATE_CODE_001"] = df["STATE_CODE_001"].astype(str).apply(lambda x: x.zfill(2))
df["COUNTY_CODE_003"] = df["COUNTY_CODE_003"].astype(str).apply(lambda x: x.zfill(3))
df["FIPS"] = df["STATE_CODE_001"] + df["COUNTY_CODE_003"]
df["FIPS"] = df["FIPS"].apply(lambda x: x.zfill(5))

In [220]:
df['in_US'] = df.apply(lambda x: in_us(x['LAT_016'],x['LONG_017']),axis=1)

In [221]:
print(df.shape[0],"instances in orginal NBI")
print(df['in_US'].count(),'intances in US')

484981 instances in orginal NBI
484130 intances in US


In [222]:
df2 = df[df['in_US'].isnull()]
df2.to_csv('C:/Users/feihu/Desktop/result/notinUS/1992_notinus.csv', index=False)

In [223]:
df['county_compare'] = np.where(df['FIPS'] == df['in_US'], True , False)
df['state'] = df['in_US'].str[:2]
df['state_compare'] = np.where(df['state'] == df['STATE_CODE_001'], True , False)
print('There are',df['county_compare'].sum(),'instances have same FIPS code')
print('There are',df.shape[0]-df['county_compare'].sum(),'instances have difference county FIPS code')
print('There are',df['state_compare'].sum(),'instances have same state FIPS code')
print('There are',df.shape[0]-df['state_compare'].sum(),'instances have difference state FIPS code')

There are 452445 instances have same FIPS code
There are 32536 instances have difference county FIPS code
There are 482437 instances have same state FIPS code
There are 2544 instances have difference state FIPS code


In [224]:
df.to_csv('C:/Users/feihu/Desktop/result/result/1992_reslut.csv', index=False)

In [225]:
df3 = df[df['county_compare'] == False]
df3.to_csv('C:/Users/feihu/Desktop/result/diffFIPS_county/1992_diffFIPS.csv', index=False)
df4 = df[df['state_compare'] == False]
df4.to_csv('C:/Users/feihu/Desktop/result/diffFIPS_state/1992_diffFIPS.csv', index=False)